# Hands-on Lab: Build an Interactive Dashboard with Ploty Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

This dashboard application contains input components such as a dropdown list and a range slider to
interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

TASK 1: Add a Launch Site Drop-down Input Component


TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown


TASK 3: Add a Range Slider to Select Payload


TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

1. Which site has the largest successful launches?
2. Which site has the highest launch success rate?
3. Which payload range(s) has the highest launch success rate?
4. Which payload range(s) has the lowest launch success rate?
5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest
   launch success rate?

### 🚀 TASK 1: Import Libraries & Load Data

First, we import required libraries and load the dataset.

In [13]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Load dataset
file_path = "spacex_launch_dash.csv"
spacex_df = pd.read_csv(file_path)

# Get min and max payload values
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


C:\Users\anupa\AppData\Local\Temp\ipykernel_13248\2975350096.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\anupa\AppData\Local\Temp\ipykernel_13248\2975350096.py:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


### 🖥 TASK 2: Create Dash App & Define Layout

Now, we create the Dash app and define the UI layout.

In [19]:
# Create Dash app
app = JupyterDash(__name__)

# App layout
app.layout = html.Div([
    # Dashboard Title
    html.H1('SpaceX Launch Records Dashboard', 
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # 📌 TASK 2.1: Dropdown for Launch Site Selection
    dcc.Dropdown(id='site-dropdown',
                 options=[{'label': 'All Sites', 'value': 'ALL'}] +
                         [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
                 value='ALL',  # Default value
                 placeholder='Select a Launch Site here',
                 searchable=True
                ),
    
    html.Br(),

    # 📌 TASK 2.2: Pie Chart for Launch Success
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # 📌 TASK 2.3: Payload Range Slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(id='payload-slider',
                    min=0, max=10000, step=1000,
                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                    value=[min_payload, max_payload]),

    html.Br(),

    # 📌 TASK 2.4: Scatter Plot for Payload vs. Success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])


C:\Users\anupa\anaconda3\Lib\site-packages\dash\dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



### 📊 TASK 3: Callback for Pie Chart

This callback function updates the Pie Chart based on the selected launch site.

In [22]:
# Callback for Pie Chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    # If 'ALL' is selected, show total successes per site
    if selected_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', 
                     title='Total Successful Launches by Site')
    else:
        # Filter data for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(filtered_df, names='class', 
                     title=f'Success vs Failure at {selected_site}')
    
    return fig


### 📈 TASK 4: Callback for Scatter Plot

This callback function updates the Scatter Plot based on:
1. Selected Launch Site
2. Payload Range

In [27]:
# Callback for Scatter Plot
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'), 
     Input('payload-slider', 'value')]
)
def update_scatter_plot(selected_site, payload_range):
    # Filter data based on payload range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                            (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
    
    # If a specific site is selected, filter further
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    # Generate scatter plot
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                     color='Booster Version Category', 
                     title='Payload vs. Launch Outcome')
    
    return fig


### 🚀 TASK 5: Run the Dash App in Jupyter Notebook


Finally, we run the Dash app inside Jupyter Notebook.

In [31]:
# Run app in Jupyter Notebook
app.run_server(mode='inline')
